In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random 
import time



In [2]:
matrix = torch.randn(32, 1, 40) 

num_neighbors = 5 
rand_idx = random.sample(range(0, 40), num_neighbors)
print(sorted(rand_idx))
rand_idx = [5, 12, 29, 30, 37]

matrix_prime = matrix[:, :, rand_idx]
print(matrix_prime.shape)


[9, 11, 12, 14, 32]
torch.Size([32, 1, 5])


In [3]:
print(torch.equal(matrix[:, :, 5],  matrix_prime[:, :, 0]))
print(torch.equal(matrix[:, :, 12], matrix_prime[:, :, 1]))
print(torch.equal(matrix[:, :, 29], matrix_prime[:, :, 2]))
print(torch.equal(matrix[:, :, 30], matrix_prime[:, :, 3]))
print(torch.equal(matrix[:, :, 37], matrix_prime[:, :, 4]))


True
True
True
True
True


In [4]:
matrix = matrix
matrix_prime = matrix_prime

distance_matrix = torch.zeros(32, 40, 5)

for i in range(32): 
   for j in range(40): 
         for k in range(5): 
            distance_matrix[i, j, k] = torch.norm(matrix[i, :, j] -  matrix_prime[i, :, k])

print(distance_matrix.shape)


torch.Size([32, 40, 5])


### Vectorization 

In [5]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random 


In [6]:
matrix = torch.randn(32, 1, 40) 

num_neighbors = 5 
rand_idx = random.sample(range(0, 40), num_neighbors)
rand_idx = [5, 12, 29, 30, 37]

matrix_prime = matrix[:, :, rand_idx]
print(matrix_prime.shape)


torch.Size([32, 1, 5])


In [7]:
import torch

def calculate_dot_product(matrix, matrix_prime):
   return torch.bmm(matrix.transpose(2, 1), matrix_prime)


def calculate_distance_matrix_N(m1, mp):
    '''Calculate the distance matrix between two input matrices'''
    norm_squared_1 = torch.sum(m1 ** 2, dim=1, keepdim=True)
    norm_squared_2 = torch.sum(mp ** 2, dim=1, keepdim=True).transpose(2, 1)
    dot_product = calculate_dot_product(m1, mp)

    norm_squared_1 = norm_squared_1.permute(0, 2, 1)  
    norm_squared_2 = norm_squared_2.permute(0, 2, 1)  

    dist_matrix = norm_squared_1 + norm_squared_2 - 2 * dot_product
    return dist_matrix

start1 = time.time() 
a = calculate_distance_matrix(matrix, matrix_prime)
print(a.shape)
end1 = time.time()
print("Vectorized", end1 - start1)

start2 = time.time()
b = torch.zeros(32, 40, 5)

for i in range(32): 
   for j in range(40): 
         for k in range(5): 
            b[i, j, k] = torch.norm(matrix[i, :, j] -  matrix_prime[i, :, k])
end2 = time.time()
print("Brute Force", end2 - start2)



NameError: name 'calculate_distance_matrix' is not defined

In [8]:
def prime_N(matrix, distance_matrix, num_nearest_neighbors, rand_idx):         
   stack_list = [] 

   for i in range(matrix.shape[0]): 
      concat_list = []
      for j in range(matrix.shape[2]): 
            if j in rand_idx: 
               indices = torch.topk(distance_matrix[i, j, :], num_nearest_neighbors, largest=False).indices
               indices_list = [rand_idx[i] for i in indices]
               nearest_neighbors = matrix[i, :, indices_list]
            else: 
               indices = torch.topk(distance_matrix[i, j, :], num_nearest_neighbors - 1, largest=False).indices
               indices_list = [j] + [rand_idx[i] for i in indices]
               nearest_neighbors = matrix[i, :, indices_list]
               
            concat_list.append(nearest_neighbors)
      
      stacked_neighbors = torch.cat(concat_list, dim=1)
      stack_list.append(stacked_neighbors)
   

   prime = torch.stack(stack_list, dim= 0)
   return prime

In [9]:
a1 = prime_N(matrix, a, 3, rand_idx)
b1 = prime_N(matrix, b, 3, rand_idx)

NameError: name 'a' is not defined

In [10]:
print(a1.shape)
print(b1.shape)

NameError: name 'a1' is not defined

In [11]:
print(a1[1, 0, :][0:3])
print(b1[1, 0, :][0:3])

NameError: name 'a1' is not defined

In [12]:
differences = a1 != b1

# Step 2: Find indices where they differ
diff_indices = differences.nonzero()
print(diff_indices)
print(diff_indices.shape)

NameError: name 'a1' is not defined

## Faiss Similarity search

In [2]:
import faiss
import numpy as np
import time

# Generate a dataset with more than 1 vector, e.g., 1000 vectors
data = np.random.rand(1000, 120).astype('float32')
print(data.shape)

index = faiss.IndexFlatL2(120)

# Add the dataset to the index
index.add(data)

# Generate a random query vector
query = np.random.rand(1000, 120).astype('float32')
k = 5  # Number of nearest neighbors to search for

start = time.time()
distances, indices = index.search(query, k)
end = time.time()

print(f"Time taken to run the function: {end - start} seconds")

# Print the indices of the k nearest neighbors
print(indices.shape)

(1000, 120)


In [1]:
import faiss
import numpy as np
import time

# Generate a dataset with more than 1 vector, e.g., 1000 vectors
data = np.random.rand(1000, 120).astype('float32')
print(data.shape)

index = faiss.IndexFlatL2(120)

# Add the dataset to the index
index.add(data)

# Generate a random query vector
query = np.random.rand(1000, 120).astype('float32')
k = 5  # Number of nearest neighbors to search for

start = time.time()
distances, indices = index.search(query, k)
end = time.time()

print(f"Time taken to run the function: {end - start} seconds")

# Print the indices of the k nearest neighbors
print(indices.shape)



(1000, 120)
Time taken to run the function: 0.0037970542907714844 seconds
(1000, 5)
